# 获取安卓&iOS基础数据

iOS 数据是修正过的

- lw_20250903_aos_gpir_cohort_onlyprofit_raw_table_by_j
- lw_20250903_ios_af_cohort_onlyprofit_fit_table_by_j


In [0]:
%sql
USE data_science.default

安卓基础数据，只是将国家分组，将applovin分成d7和d28。纯利

In [0]:
%sql
CREATE OR REPLACE TABLE lw_20250903_aos_gpir_cohort_onlyprofit_raw_table_by_j as
SELECT
	'com.fun.lastwar.gp' as app_package,	
	COALESCE(cg.country_group, 'other') AS country_group,
	case
	when roi.mediasource = 'applovin_int' and pub.campaign_name like '%D7%' then 'applovin_int_d7'
	when roi.mediasource = 'applovin_int' and pub.campaign_name like '%D28%' then 'applovin_int_d28'
	WHEN mediasource in ('Organic', 'organic') THEN 'Organic'
	else roi.mediasource end as mediasource,
	'aos_gpir_cohort_onlyprofit_raw' as tag,
    roi.install_day,
	sum(cost_value_usd) as cost,
	SUM(revenue_h24) AS revenue_d1,
	SUM(revenue_h72) AS revenue_d3,
	SUM(revenue_h168) AS revenue_d7,
	SUM(revenue_cohort_d14) AS revenue_d14,
	SUM(revenue_cohort_d30) AS revenue_d30,
	SUM(revenue_cohort_d60) AS revenue_d60,
	SUM(revenue_cohort_d90) AS revenue_d90,
	SUM(revenue_cohort_d120) AS revenue_d120,
	SUM(revenue_cohort_d135) AS revenue_d135,
	SUM(revenue_cohort_d150) AS revenue_d150
FROM
	marketing.attribution.dws_overseas_gpir_roi_profit roi
	LEFT JOIN lw_country_group_table_by_j_20250703 cg ON roi.country = cg.country
    LEFT JOIN (
		SELECT campaign_id, MAX(campaign_name) AS campaign_name
		FROM prodb.public.applovin_campaign_info_new
		GROUP BY campaign_id
	) pub ON 
    roi.campaign_id = pub.campaign_id
WHERE
	roi.app_package in ('com.fun.lastwar.gp','com.fun.lastwar.vn.gp')
GROUP BY
	1,2,3,4,5
;

iOS基础数据，同安卓。纯利。
注意，由于这是AF归因结果，所以是模糊归因，里面google等媒体数据不准。


In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_ios_af_cohort_onlyprofit_raw_view_by_j as
SELECT
	'id6448786147' as app_package,
	roi.install_day,
	COALESCE(cg.country_group, 'other') AS country_group,
	CASE
		WHEN mediasource = 'applovin_int' AND UPPER(campaign_name) LIKE '%D7%' THEN 'applovin_int_d7'
		WHEN mediasource = 'applovin_int' AND UPPER(campaign_name) LIKE '%D28%' THEN 'applovin_int_d28'
        WHEN mediasource in ('Organic', 'organic') THEN 'Organic'
		ELSE mediasource
	END as mediasource,
	'ALL' AS ad_type,
	SUM(cost_value_usd) AS cost,
	SUM(revenue_h24) AS revenue_d1,
	SUM(revenue_h72) AS revenue_d3,
	SUM(revenue_h168) AS revenue_d7,
	SUM(revenue_cohort_d14) AS revenue_d14,
	SUM(revenue_cohort_d30) AS revenue_d30,
	SUM(revenue_cohort_d60) AS revenue_d60,
	SUM(revenue_cohort_d90) AS revenue_d90,
	SUM(revenue_cohort_d120) AS revenue_d120,
    SUM(revenue_cohort_d135) AS revenue_d135,
	SUM(revenue_cohort_d150) AS revenue_d150
FROM
	marketing.attribution.dws_overseas_roi_profit roi
	LEFT JOIN lw_country_group_table_by_j_20250703 cg ON roi.country = cg.country
	LEFT JOIN (
		SELECT
			campaign_id,
			MAX(campaign_name) AS campaign_name
		FROM
			prodb.public.applovin_campaign_info_new
		GROUP BY
			campaign_id
	) pub ON roi.campaign_id = pub.campaign_id
WHERE
	roi.app_package in ('id6448786147', 'id6736925794')
GROUP BY
	roi.install_day,
	COALESCE(cg.country_group, 'other'),
	CASE
		WHEN mediasource = 'applovin_int' AND UPPER(campaign_name) LIKE '%D7%' THEN 'applovin_int_d7'
		WHEN mediasource = 'applovin_int' AND UPPER(campaign_name) LIKE '%D28%' THEN 'applovin_int_d28'
        WHEN mediasource in ('Organic', 'organic') THEN 'Organic'
		ELSE mediasource
	END,
	ad_type;

iOS 数据修正：
将google按照所有媒体均值进行估计；
按照新的google收入，重新计算自然量。
注意，这里是table，不是view，为了解决后续可能得循环引用问题。


In [0]:
%sql
CREATE OR REPLACE TABLE lw_20250903_ios_af_cohort_onlyprofit_fix_view_by_j AS
WITH base_data AS (
	SELECT
		app_package,
		install_day,
		country_group,
		mediasource,
		ad_type,
		cost,
		revenue_d1,
		revenue_d3,
		revenue_d7,
		revenue_d14,
		revenue_d30,
		revenue_d60,
		revenue_d90,
		revenue_d120,
        revenue_d135,
		revenue_d150
	FROM lw_20250903_ios_af_cohort_onlyprofit_raw_view_by_j
),
paid_media_totals AS (
	-- 计算付费媒体的总花费（用于计算Google的收入比例）
	SELECT
		app_package,
		install_day,
		country_group,
		SUM(cost) AS total_paid_cost,
		SUM(revenue_d1) AS total_paid_revenue_d1,
		SUM(revenue_d3) AS total_paid_revenue_d3,
		SUM(revenue_d7) AS total_paid_revenue_d7,
		SUM(revenue_d14) AS total_paid_revenue_d14,
		SUM(revenue_d30) AS total_paid_revenue_d30,
		SUM(revenue_d60) AS total_paid_revenue_d60,
		SUM(revenue_d90) AS total_paid_revenue_d90,
		SUM(revenue_d120) AS total_paid_revenue_d120,
        SUM(revenue_d135) AS total_paid_revenue_d135,
		SUM(revenue_d150) AS total_paid_revenue_d150
	FROM base_data
	WHERE mediasource != 'Organic'
		AND mediasource != 'googleadwords_int'
	GROUP BY
		app_package,
		install_day,
		country_group
),
daily_totals AS (
	-- 计算每天的总收入（用于后续自然量修正）
	SELECT
		app_package,
		install_day,
		country_group,
		SUM(revenue_d1) AS total_revenue_d1,
		SUM(revenue_d3) AS total_revenue_d3,
		SUM(revenue_d7) AS total_revenue_d7,
		SUM(revenue_d14) AS total_revenue_d14,
		SUM(revenue_d30) AS total_revenue_d30,
		SUM(revenue_d60) AS total_revenue_d60,
		SUM(revenue_d90) AS total_revenue_d90,
		SUM(revenue_d120) AS total_revenue_d120,
        SUM(revenue_d135) AS total_revenue_d135,
		SUM(revenue_d150) AS total_revenue_d150
	FROM base_data
	GROUP BY
		app_package,
		install_day,
		country_group
),
adjusted_media_data AS (
	-- 调整Google的收入，其他媒体保持不变
	SELECT
		bd.app_package,
		bd.install_day,
		bd.country_group,
		bd.mediasource,
		bd.ad_type,
		bd.cost,
		-- 原始收入
		bd.revenue_d1 AS original_revenue_d1,
		bd.revenue_d3 AS original_revenue_d3,
		bd.revenue_d7 AS original_revenue_d7,
		bd.revenue_d14 AS original_revenue_d14,
		bd.revenue_d30 AS original_revenue_d30,
		bd.revenue_d60 AS original_revenue_d60,
		bd.revenue_d90 AS original_revenue_d90,
		bd.revenue_d120 AS original_revenue_d120,
        bd.revenue_d135 AS original_revenue_d135,
		bd.revenue_d150 AS original_revenue_d150,
		-- 调整后收入
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d1 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d1
		END AS adjusted_revenue_d1,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d3 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d3
		END AS adjusted_revenue_d3,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d7 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d7
		END AS adjusted_revenue_d7,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d14 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d14
		END AS adjusted_revenue_d14,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d30 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d30
		END AS adjusted_revenue_d30,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d60 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d60
		END AS adjusted_revenue_d60,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d90 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d90
		END AS adjusted_revenue_d90,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d120 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d120
		END AS adjusted_revenue_d120,
        CASE
            WHEN bd.mediasource = 'googleadwords_int' THEN
                CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
                    pmt.total_paid_revenue_d135 * (bd.cost / pmt.total_paid_cost)
                ELSE 0 END
            ELSE bd.revenue_d135
        END AS adjusted_revenue_d135,
		CASE
			WHEN bd.mediasource = 'googleadwords_int' THEN
				CASE WHEN COALESCE(pmt.total_paid_cost, 0) > 0 THEN
					pmt.total_paid_revenue_d150 * (bd.cost / pmt.total_paid_cost)
				ELSE 0 END
			ELSE bd.revenue_d150
		END AS adjusted_revenue_d150
	FROM base_data bd
	LEFT JOIN paid_media_totals pmt ON bd.app_package = pmt.app_package
									AND bd.install_day = pmt.install_day
									AND bd.country_group = pmt.country_group
),
adjusted_paid_totals AS (
	-- 计算调整后的付费媒体总收入
	SELECT
		app_package,
		install_day,
		country_group,
		SUM(adjusted_revenue_d1) AS total_adjusted_paid_revenue_d1,
		SUM(adjusted_revenue_d3) AS total_adjusted_paid_revenue_d3,
		SUM(adjusted_revenue_d7) AS total_adjusted_paid_revenue_d7,
		SUM(adjusted_revenue_d14) AS total_adjusted_paid_revenue_d14,
		SUM(adjusted_revenue_d30) AS total_adjusted_paid_revenue_d30,
		SUM(adjusted_revenue_d60) AS total_adjusted_paid_revenue_d60,
		SUM(adjusted_revenue_d90) AS total_adjusted_paid_revenue_d90,
		SUM(adjusted_revenue_d120) AS total_adjusted_paid_revenue_d120,
        SUM(adjusted_revenue_d135) AS total_adjusted_paid_revenue_d135,
		SUM(adjusted_revenue_d150) AS total_adjusted_paid_revenue_d150
	FROM adjusted_media_data
	WHERE mediasource != 'Organic'
	GROUP BY
		app_package,
		install_day,
		country_group
)
-- 最终结果：付费媒体使用调整后收入，自然量使用修正后收入
SELECT
	amd.app_package,
	amd.install_day,
	amd.country_group,
	amd.mediasource,
	amd.ad_type,
	amd.cost,
	-- 对于自然量，使用修正后的收入；对于付费媒体，使用调整后的收入
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d1 - COALESCE(apt.total_adjusted_paid_revenue_d1, 0))
		ELSE amd.adjusted_revenue_d1
	END AS revenue_d1,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d3 - COALESCE(apt.total_adjusted_paid_revenue_d3, 0))
		ELSE amd.adjusted_revenue_d3
	END AS revenue_d3,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d7 - COALESCE(apt.total_adjusted_paid_revenue_d7, 0))
		ELSE amd.adjusted_revenue_d7
	END AS revenue_d7,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d14 - COALESCE(apt.total_adjusted_paid_revenue_d14, 0))
		ELSE amd.adjusted_revenue_d14
	END AS revenue_d14,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d30 - COALESCE(apt.total_adjusted_paid_revenue_d30, 0))
		ELSE amd.adjusted_revenue_d30
	END AS revenue_d30,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d60 - COALESCE(apt.total_adjusted_paid_revenue_d60, 0))
		ELSE amd.adjusted_revenue_d60
	END AS revenue_d60,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d90 - COALESCE(apt.total_adjusted_paid_revenue_d90, 0))
		ELSE amd.adjusted_revenue_d90
	END AS revenue_d90,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d120 - COALESCE(apt.total_adjusted_paid_revenue_d120, 0))
		ELSE amd.adjusted_revenue_d120
	END AS revenue_d120,
    CASE
        WHEN amd.mediasource = 'Organic' THEN
            GREATEST(0, dt.total_revenue_d135 - COALESCE(apt.total_adjusted_paid_revenue_d135, 0))
        ELSE amd.adjusted_revenue_d135
    END AS revenue_d135,
	CASE
		WHEN amd.mediasource = 'Organic' THEN
			GREATEST(0, dt.total_revenue_d150 - COALESCE(apt.total_adjusted_paid_revenue_d150, 0))
		ELSE amd.adjusted_revenue_d150
	END AS revenue_d150
FROM adjusted_media_data amd
LEFT JOIN daily_totals dt ON amd.app_package = dt.app_package
    AND amd.install_day = dt.install_day
    AND amd.country_group = dt.country_group
LEFT JOIN adjusted_paid_totals apt ON amd.app_package = apt.app_package
    AND amd.install_day = apt.install_day
    AND amd.country_group = apt.country_group
ORDER BY
	amd.app_package,
	amd.install_day,
	amd.country_group,
	amd.mediasource
;

iOS数据拟合：用上面修正数据作为基础，几个主要媒体乘以贝叶斯拟合倍率；
然后再重新计算自然量。


In [0]:
%sql
CREATE OR REPLACE TABLE lw_20250903_ios_af_cohort_onlyprofit_fit_table_by_j AS
WITH base_data AS (
	SELECT
		app_package,
		install_day,
		country_group,
		mediasource,
		cost,
        revenue_d1,
        revenue_d3,
		revenue_d7,
        revenue_d14,
        revenue_d30,
        revenue_d60,
        revenue_d90,
		revenue_d120,
        revenue_d135,
        revenue_d150
	FROM lw_20250903_ios_af_cohort_onlyprofit_fix_view_by_j
	WHERE mediasource != 'Organic'
),
original_totals AS (
	SELECT
		app_package,
		install_day,
		country_group,
        SUM(revenue_d1) AS total_original_revenue_d1,
        SUM(revenue_d3) AS total_original_revenue_d3,
		SUM(revenue_d7) AS total_original_revenue_d7,
        SUM(revenue_d14) AS total_original_revenue_d14,
        SUM(revenue_d30) AS total_original_revenue_d30,
        SUM(revenue_d60) AS total_original_revenue_d60,
        SUM(revenue_d90) AS total_original_revenue_d90,
		SUM(revenue_d120) AS total_original_revenue_d120,
        SUM(revenue_d135) AS total_original_revenue_d135,
        SUM(revenue_d150) AS total_original_revenue_d150
	FROM lw_20250903_ios_af_cohort_onlyprofit_fix_view_by_j
	GROUP BY
		app_package,
		install_day,
		country_group
),
bayesian_results AS (
	-- 获取拟合系数和自然量收入
	SELECT 
		country_group,
		organic_revenue,
		applovin_int_d7_coeff,
		applovin_int_d28_coeff,
		facebook_ads_coeff,
		moloco_int_coeff,
		bytedanceglobal_int_coeff,
		tag
	FROM lw_20250703_ios_bayesian_result_by_j
	WHERE tag IN ('20250808_20')
),
fitted_paid_media AS (
	-- 对付费媒体应用拟合系数
	SELECT
		bd.app_package,
		bd.install_day,
		bd.country_group,
		bd.mediasource,
		bd.cost,
		br.tag,
		CASE
			WHEN bd.mediasource = 'applovin_int_d7' THEN 
				bd.revenue_d1 * COALESCE(br.applovin_int_d7_coeff, 1.0)
			WHEN bd.mediasource = 'applovin_int_d28' THEN 
				bd.revenue_d1 * COALESCE(br.applovin_int_d28_coeff, 1.0)
			WHEN bd.mediasource = 'Facebook Ads' THEN 
				bd.revenue_d1 * COALESCE(br.facebook_ads_coeff, 1.0)
			WHEN bd.mediasource = 'moloco_int' THEN 
				bd.revenue_d1 * COALESCE(br.moloco_int_coeff, 1.0)
			WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
				bd.revenue_d1 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
			ELSE bd.revenue_d1
		END AS fitted_revenue_d1,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d3 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d3 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d3 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d3 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d3 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d3
        END AS fitted_revenue_d3,
		CASE
			WHEN bd.mediasource = 'applovin_int_d7' THEN 
				bd.revenue_d7 * COALESCE(br.applovin_int_d7_coeff, 1.0)
			WHEN bd.mediasource = 'applovin_int_d28' THEN 
				bd.revenue_d7 * COALESCE(br.applovin_int_d28_coeff, 1.0)
			WHEN bd.mediasource = 'Facebook Ads' THEN 
				bd.revenue_d7 * COALESCE(br.facebook_ads_coeff, 1.0)
			WHEN bd.mediasource = 'moloco_int' THEN 
				bd.revenue_d7 * COALESCE(br.moloco_int_coeff, 1.0)
			WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
				bd.revenue_d7 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
			ELSE bd.revenue_d7
		END AS fitted_revenue_d7,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d14 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d14 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d14 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d14 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d14 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d14
        END AS fitted_revenue_d14,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d30 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d30 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d30 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d30 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d30 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d30
        END AS fitted_revenue_d30,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d60 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d60 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d60 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d60 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d60 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d60
        END AS fitted_revenue_d60,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d90 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d90 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d90 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d90 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d90 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d90
        END AS fitted_revenue_d90,
		CASE
			WHEN bd.mediasource = 'applovin_int_d7' THEN 
				bd.revenue_d120 * COALESCE(br.applovin_int_d7_coeff, 1.0)
			WHEN bd.mediasource = 'applovin_int_d28' THEN 
				bd.revenue_d120 * COALESCE(br.applovin_int_d28_coeff, 1.0)
			WHEN bd.mediasource = 'Facebook Ads' THEN 
				bd.revenue_d120 * COALESCE(br.facebook_ads_coeff, 1.0)
			WHEN bd.mediasource = 'moloco_int' THEN 
				bd.revenue_d120 * COALESCE(br.moloco_int_coeff, 1.0)
			WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
				bd.revenue_d120 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
			ELSE bd.revenue_d120
		END AS fitted_revenue_d120,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d135 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d135 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d135 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d135 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d135 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d135
        END AS fitted_revenue_d135,
        CASE
            WHEN bd.mediasource = 'applovin_int_d7' THEN 
                bd.revenue_d150 * COALESCE(br.applovin_int_d7_coeff, 1.0)
            WHEN bd.mediasource = 'applovin_int_d28' THEN 
                bd.revenue_d150 * COALESCE(br.applovin_int_d28_coeff, 1.0)
            WHEN bd.mediasource = 'Facebook Ads' THEN 
                bd.revenue_d150 * COALESCE(br.facebook_ads_coeff, 1.0)
            WHEN bd.mediasource = 'moloco_int' THEN 
                bd.revenue_d150 * COALESCE(br.moloco_int_coeff, 1.0)
            WHEN bd.mediasource = 'bytedanceglobal_int' THEN 
                bd.revenue_d150 * COALESCE(br.bytedanceglobal_int_coeff, 1.0)
            ELSE bd.revenue_d150
        END AS fitted_revenue_d150
	FROM base_data bd
	LEFT JOIN bayesian_results br ON bd.country_group = br.country_group
),
fitted_paid_totals AS (
	-- 计算调整后的付费媒体总收入（用于计算120日自然量）
	SELECT
		app_package,
		install_day,
		country_group,
		tag,
        SUM(fitted_revenue_d1) AS total_fitted_paid_revenue_d1,
        SUM(fitted_revenue_d3) AS total_fitted_paid_revenue_d3,
		SUM(fitted_revenue_d7) AS total_fitted_paid_revenue_d7,
        SUM(fitted_revenue_d14) AS total_fitted_paid_revenue_d14,
        SUM(fitted_revenue_d30) AS total_fitted_paid_revenue_d30,
        SUM(fitted_revenue_d60) AS total_fitted_paid_revenue_d60,
        SUM(fitted_revenue_d90) AS total_fitted_paid_revenue_d90,
		SUM(fitted_revenue_d120) AS total_fitted_paid_revenue_d120,
        SUM(fitted_revenue_d135) AS total_fitted_paid_revenue_d135,
        SUM(fitted_revenue_d150) AS total_fitted_paid_revenue_d150
	FROM fitted_paid_media
	GROUP BY
		app_package,
		install_day,
		country_group,
		tag
),
organic_data AS (
	-- 生成自然量数据
	SELECT DISTINCT
		bd.app_package,
		bd.install_day,
		bd.country_group,
		'Organic' as mediasource,
		0.0 as cost,
		br.tag,
		GREATEST(0, ot.total_original_revenue_d1 - COALESCE(fpt.total_fitted_paid_revenue_d1, 0)) as fitted_revenue_d1,
        GREATEST(0, ot.total_original_revenue_d3 - COALESCE(fpt.total_fitted_paid_revenue_d3, 0)) as fitted_revenue_d3,
        GREATEST(0, ot.total_original_revenue_d7 - COALESCE(fpt.total_fitted_paid_revenue_d7, 0)) as fitted_revenue_d7,
        GREATEST(0, ot.total_original_revenue_d14 - COALESCE(fpt.total_fitted_paid_revenue_d14, 0)) as fitted_revenue_d14,
        GREATEST(0, ot.total_original_revenue_d30 - COALESCE(fpt.total_fitted_paid_revenue_d30, 0)) as fitted_revenue_d30,
        GREATEST(0, ot.total_original_revenue_d60 - COALESCE(fpt.total_fitted_paid_revenue_d60, 0)) as fitted_revenue_d60,
        GREATEST(0, ot.total_original_revenue_d90 - COALESCE(fpt.total_fitted_paid_revenue_d90, 0)) as fitted_revenue_d90,
		GREATEST(0, ot.total_original_revenue_d120 - COALESCE(fpt.total_fitted_paid_revenue_d120, 0)) as fitted_revenue_d120,
        GREATEST(0, ot.total_original_revenue_d135 - COALESCE(fpt.total_fitted_paid_revenue_d135, 0)) as fitted_revenue_d135,
        GREATEST(0, ot.total_original_revenue_d150 - COALESCE(fpt.total_fitted_paid_revenue_d150, 0)) as fitted_revenue_d150
	FROM (
		SELECT DISTINCT app_package, install_day, country_group 
		FROM base_data
	) bd
	LEFT JOIN bayesian_results br ON bd.country_group = br.country_group
	LEFT JOIN original_totals ot ON bd.app_package = ot.app_package
        AND bd.install_day = ot.install_day
        AND bd.country_group = ot.country_group
	LEFT JOIN fitted_paid_totals fpt ON bd.app_package = fpt.app_package
        AND bd.install_day = fpt.install_day
        AND bd.country_group = fpt.country_group
        AND br.tag = fpt.tag
	WHERE br.organic_revenue IS NOT NULL
)
-- 合并付费媒体和自然量数据
SELECT
	app_package,
	country_group,
	mediasource,
	tag,
    install_day,
	cost,
    fitted_revenue_d1 as revenue_d1,
    fitted_revenue_d3 as revenue_d3,
	fitted_revenue_d7 as revenue_d7,
    fitted_revenue_d14 as revenue_d14,
    fitted_revenue_d30 as revenue_d30,
    fitted_revenue_d60 as revenue_d60,
    fitted_revenue_d90 as revenue_d90,
	fitted_revenue_d120 as revenue_d120,
    fitted_revenue_d135 as revenue_d135,
    fitted_revenue_d150 as revenue_d150
FROM fitted_paid_media

UNION ALL

SELECT
	app_package,
	country_group,
	mediasource,
	tag,
    install_day,
	cost,
	fitted_revenue_d1 as revenue_d1,
    fitted_revenue_d3 as revenue_d3,
	fitted_revenue_d7 as revenue_d7,
    fitted_revenue_d14 as revenue_d14,
    fitted_revenue_d30 as revenue_d30,
    fitted_revenue_d60 as revenue_d60,
    fitted_revenue_d90 as revenue_d90,
	fitted_revenue_d120 as revenue_d120,
    fitted_revenue_d135 as revenue_d135,
    fitted_revenue_d150 as revenue_d150
FROM organic_data

ORDER BY
	tag,
	app_package,
	install_day,
	country_group,
	mediasource
;